# 제품 이상여부 판별 프로젝트


## 데이터 불러오기


### 필수 라이브러리


In [50]:
#!pip install xgboost==2.1.1


[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
from pprint import pprint

import optuna
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

### 데이터 읽어오기


In [2]:
import pandas as pd

THRESHOLD = 0.3
RANDOM_STATE = 110

train_data = pd.read_csv("./data/train_data_0827.csv")
test_data = pd.read_csv("./data/test_data_0827.csv")

In [3]:
# dam, fill1, fill2 공통 변수
var_dam_fill = [
    'Receip_No_encoded',
    'Equipment_same_num',
    'PalletID_Collect_Result_encoded',
    'Production_Qty_Collect_Result',
    'WorkMode Collect Result'
]

# 전체 공통 변수
### train
var_all_train = [
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

### test
var_all_test = [
    'Set ID',
    'target',
    'model_suffix_encoded',
    'cleaned_workorder_encoded'
]

In [4]:
# '_Dam'을 포함하는 변수 선택
dam_variables = [var for var in train_data.columns if '_Dam' in var]

# train
final_columns_train = var_dam_fill + var_all_train + dam_variables
train_data_dam = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + dam_variables
test_data_dam = test_data[final_columns_test]

In [5]:
# '_Fill1'을 포함하는 변수 선택
fill1_variables = [var for var in train_data.columns if '_Fill1' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill1_variables
train_data_fill1 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill1_variables
test_data_fill1 = test_data[final_columns_test]

In [6]:
# '_Fill2'을 포함하는 변수 선택
fill2_variables = [var for var in train_data.columns if '_Fill2' in var]

# train
final_columns_train = var_dam_fill + var_all_train + fill2_variables
train_data_fill2 = train_data[final_columns_train]

# test 
final_columns_test = var_dam_fill + var_all_test + fill2_variables
test_data_fill2 = test_data[final_columns_test]

In [7]:
# '_AutoClave'을 포함하는 변수 선택
autoclave_variables = [var for var in train_data.columns if '_AutoClave' in var]

# train
final_columns_train = var_all_train + autoclave_variables
train_data_autoclave = train_data[final_columns_train]

# test 
final_columns_test = var_all_test + autoclave_variables
test_data_autoclave = test_data[final_columns_test]

In [8]:
# 각 DataFrame의 칼럼 수 계산
num_columns_train_data = train_data.shape[1]
num_columns_train_data_dam = train_data_dam.shape[1]
num_columns_train_data_autoclave = train_data_autoclave.shape[1]
num_columns_train_data_fill1 = train_data_fill1.shape[1]
num_columns_train_data_fill2 = train_data_fill2.shape[1]

num_columns_test_data = test_data.shape[1]
num_columns_test_data_dam = test_data_dam.shape[1]
num_columns_test_data_autoclave = test_data_autoclave.shape[1]
num_columns_test_data_fill1 = test_data_fill1.shape[1]
num_columns_test_data_fill2 = test_data_fill2.shape[1]

# 각 DataFrame의 칼럼 수 출력
print("----train data-----")
print(f"train_data DataFrame의 칼럼 수: {num_columns_train_data}")
print(f"train_data_dam DataFrame의 칼럼 수: {num_columns_train_data_dam}")
print(f"train_data_autoclave DataFrame의 칼럼 수: {num_columns_train_data_autoclave}")
print(f"train_data_fill1 DataFrame의 칼럼 수: {num_columns_train_data_fill1}")
print(f"train_data_fill2 DataFrame의 칼럼 수: {num_columns_train_data_fill2}")
print("----test data-----")
print(f"test_data DataFrame의 칼럼 수: {num_columns_test_data}")
print(f"test_data_dam DataFrame의 칼럼 수: {num_columns_test_data_dam}")
print(f"test_data_autoclave DataFrame의 칼럼 수: {num_columns_test_data_autoclave}")
print(f"test_data_fill1 DataFrame의 칼럼 수: {num_columns_test_data_fill1}")
print(f"test_data_fill2 DataFrame의 칼럼 수: {num_columns_test_data_fill2}")

----train data-----
train_data DataFrame의 칼럼 수: 40
train_data_dam DataFrame의 칼럼 수: 23
train_data_autoclave DataFrame의 칼럼 수: 8
train_data_fill1 DataFrame의 칼럼 수: 14
train_data_fill2 DataFrame의 칼럼 수: 14
----test data-----
test_data DataFrame의 칼럼 수: 41
test_data_dam DataFrame의 칼럼 수: 24
test_data_autoclave DataFrame의 칼럼 수: 9
test_data_fill1 DataFrame의 칼럼 수: 15
test_data_fill2 DataFrame의 칼럼 수: 15


---

## 모델링

### 모델 정의

In [9]:
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.ensemble import VotingClassifier

# 스레드홀드 설정
THRESHOLD = 0.3

# 모델 설정 및 하이퍼파라미터
models = {
    'et': ExtraTreesClassifier(),
    'rf': RandomForestClassifier(),
    'cat': CatBoostClassifier(),
    'lgbm': LGBMClassifier(),
    'xgb': XGBClassifier(),
    'dt': DecisionTreeClassifier(),
    'ada': AdaBoostClassifier()
}

def train_and_evaluate_model(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return
    
    # 데이터셋 분할
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(x_train, y_train)

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    # 예측
    y_val_pred_proba = model.predict_proba(x_val)[:, 1]  # 양성 클래스 확률
    y_val_pred = (y_val_pred_proba >= THRESHOLD).astype(int)  # 스레드홀드에 따른 예측

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'{model_name} 모델이 {data_name} 데이터로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return model  # 학습된 모델 반환

def fit_all_train_data_function(model_name, data, **params):
    if model_name not in models:
        print(f"{model_name}은(는) 지원되지 않는 모델입니다.")
        return None  # 지원되지 않는 모델일 경우 None 반환
    
    # 모델 선택
    model = models[model_name].__class__()  # 새로운 모델 인스턴스 생성

    # 하이퍼파라미터 설정
    model.set_params(**params)

    # 모델 학습
    model.fit(data.drop("target", axis=1), data["target"].map({'Normal': 0, 'AbNormal': 1}))

    # 데이터 이름을 자동으로 추출하기 위한 래퍼 함수
    data_name = [name for name in globals() if globals()[name] is data][0]

    print(f'{model_name} 모델이 {data_name} 데이터로 학습 완료')
    return model  # 학습된 모델 반환

def voting_function(data, estimators, voting='hard', threshold=0.5):
    # 데이터셋 분할 # voting='hard'일 경우 threshold는 사용되지 않음
    x_train, x_val, y_train, y_val = train_test_split(
        data.drop("target", axis=1),
        data["target"].map({'Normal': 0, 'AbNormal': 1}),
        test_size=0.2,
        shuffle=True,
        random_state=RANDOM_STATE,
    )

    # VotingClassifier 설정
    voting_clf = VotingClassifier(estimators=estimators, voting=voting)

    # 모델 학습
    voting_clf.fit(x_train, y_train)

    if voting == 'soft':
        # 소프트 보팅의 경우 확률 예측
        y_val_pred_proba = voting_clf.predict_proba(x_val)[:, 1]
        y_val_pred = (y_val_pred_proba >= threshold).astype(int)
    else:
        # 하드 보팅의 경우 직접 예측
        y_val_pred = voting_clf.predict(x_val)

    # 평가지표 계산
    f1 = f1_score(y_val, y_val_pred, average="binary")
    accuracy = accuracy_score(y_val, y_val_pred)
    precision = precision_score(y_val, y_val_pred, zero_division=0)
    recall = recall_score(y_val, y_val_pred)
    conf_matrix = confusion_matrix(y_val, y_val_pred)
    
    # 결과 출력
    print(f'Voting Classifier로 학습한 결과:')
    print(f'F1 Score: {f1}')
    print('---')
    print('Confusion Matrix:')
    print(conf_matrix)
    print('---')
    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print('\n')

    return voting_clf  # 학습된 VotingClassifier 반환

def voting(preds_or_probs, method='soft', threshold=0.3):
    """
    하드 보팅 또는 소프트 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds_or_probs (list of np.array): 각 모델의 예측 배열 리스트 (하드 보팅) 또는 예측 확률 배열 리스트 (소프트 보팅)
    method (str): 'soft' 또는 'hard' 보팅 방법 선택
    threshold (float): 소프트 보팅 시 예측을 양성으로 간주할 확률 임계값

    Returns:
    np.array: 최종 예측 결과
    """
    if method == 'soft':
        # 소프트 보팅: 각 모델의 확률 평균 계산
        soft_voting_probs = np.mean(preds_or_probs, axis=0)
        # 최종 예측: 평균 확률에 대해 스레드 홀드 적용
        final_predictions = (soft_voting_probs >= threshold).astype(int)
    elif method == 'hard':
        # 하드 보팅: 각 모델의 예측을 모아서 다수결 원칙 적용
        preds = np.array(preds_or_probs)
        final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    else:
        raise ValueError("method 인자는 'soft' 또는 'hard'여야 합니다.")
    
    return final_predictions

### 모델 학습

Dam 모델

In [16]:
train_model_Dam = train_and_evaluate_model(
    'xgb', train_data_dam
    , n_estimators = 1244
    , learning_rate = 0.1258535425769987
    , max_depth = 26
    , alpha = 2.1820842842359597e-06
    , gamma = 0.00010809657684921935
    , reg_alpha = 0.5844029076359536
    , reg_lambda = 0.4748752246073433
    , colsample_bytree = 0.9607659760060685
    , subsample = 0.7147741317935203
    , objective = 'binary:logistic'
    , tree_method = 'exact'
    , random_state=RANDOM_STATE
)

xgb 모델이 train_data_dam 데이터로 학습한 결과:
F1 Score: 0.21862348178137653
---
Confusion Matrix:
[[7442  220]
 [ 359   81]]
---
Accuracy: 0.9285361639101456
Precision: 0.2691029900332226
Recall: 0.18409090909090908




AutoClave 모델

In [17]:
train_model_AutoClave = train_and_evaluate_model(
    'xgb', train_data_autoclave,
    n_estimators = 1152, 
    learning_rate = 0.02466611382982541, 
    max_depth = 29, 
    alpha = 2.9180083404308157e-05, 
    gamma = 0.00012667501319666823, 
    reg_alpha = 0.6903592486292155, 
    reg_lambda = 0.5638873235014423, 
    colsample_bytree = 0.9432782030604233, 
    subsample = 0.19192246128663584,
    objective = 'binary:logistic',  # 이진 분류
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

xgb 모델이 train_data_autoclave 데이터로 학습한 결과:
F1 Score: 0.24687933425797504
---
Confusion Matrix:
[[7470  192]
 [ 351   89]]
---
Accuracy: 0.9329795112317946
Precision: 0.3167259786476868
Recall: 0.20227272727272727




Fill1 모델

In [18]:
train_model_Fill1 = train_and_evaluate_model(
    'xgb', train_data_fill1,
    n_estimators = 1899, 
    learning_rate = 0.011878583548993711, 
    max_depth = 12, 
    alpha = 0.004515243354832891,
    gamma = 0.0015693650802180896,
    reg_alpha = 0.7484424912256998, 
    reg_lambda = 0.27164326303977143, 
    colsample_bytree = 0.7901385059430825,
    subsample = 0.9924662032617025,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

xgb 모델이 train_data_fill1 데이터로 학습한 결과:
F1 Score: 0.20890937019969277
---
Confusion Matrix:
[[7519  143]
 [ 372   68]]
---
Accuracy: 0.9364354480375215
Precision: 0.3222748815165877
Recall: 0.15454545454545454




Fill2 모델

In [19]:
train_model_Fill2 = train_and_evaluate_model(
    'xgb', train_data_fill2,
    n_estimators = 1162, 
    learning_rate = 0.014523070494025153, 
    max_depth = 8, 
    alpha = 0.00012198482017902725, 
    gamma = 0.001236902841680112, 
    reg_alpha = 0.7331637000614692, 
    reg_lambda = 0.5237223061096699, 
    colsample_bytree = 0.8250374170841293, 
    subsample = 0.31906427054137687,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

xgb 모델이 train_data_fill2 데이터로 학습한 결과:
F1 Score: 0.21731748726655348
---
Confusion Matrix:
[[7577   85]
 [ 376   64]]
---
Accuracy: 0.9431004690199951
Precision: 0.42953020134228187
Recall: 0.14545454545454545




전체공정 모델

In [20]:
train_model_All = train_and_evaluate_model(
    'xgb', train_data,
    n_estimators = 2427,
    learning_rate = 0.010774204513905965, 
    max_depth = 17, 
    alpha = 0.0005233654110538582, 
    gamma = 5.551445919277608e-05, 
    reg_alpha = 0.9652805882189326, 
    reg_lambda = 0.3542856398135083, 
    colsample_bytree = 0.9094884645797131, 
    subsample = 0.1733751790853043,
    objective = 'binary:logistic',  # 이진 분류
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

xgb 모델이 train_data 데이터로 학습한 결과:
F1 Score: 0.2608695652173913
---
Confusion Matrix:
[[7542  120]
 [ 356   84]]
---
Accuracy: 0.9412490743026414
Precision: 0.4117647058823529
Recall: 0.19090909090909092




soft voting

In [21]:
# VotingClassifier 사용 
estimators = [
      ('dam', train_model_Dam)
    , ('autoclave', train_model_AutoClave)
    , ('fill1', train_model_Fill1)
    , ('fill2', train_model_Fill2)
    # , ('all', train_model_All)
]

# VotingClassifier 학습 및 평가
# voting_clf_hard = voting_function(train_data, estimators, voting='hard')
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.3)

Voting Classifier로 학습한 결과:
F1 Score: 0.23529411764705888
---
Confusion Matrix:
[[7532  130]
 [ 364   76]]
---
Accuracy: 0.9390274006418169
Precision: 0.36893203883495146
Recall: 0.17272727272727273




In [22]:
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.28)

Voting Classifier로 학습한 결과:
F1 Score: 0.22950819672131145
---
Confusion Matrix:
[[7508  154]
 [ 363   77]]
---
Accuracy: 0.936188595408541
Precision: 0.3333333333333333
Recall: 0.175




In [23]:
voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.32)

Voting Classifier로 학습한 결과:
F1 Score: 0.22544283413848631
---
Confusion Matrix:
[[7551  111]
 [ 370   70]]
---
Accuracy: 0.9406319427301901
Precision: 0.3867403314917127
Recall: 0.1590909090909091




In [69]:
# voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.24)

In [70]:
# voting_clf_soft = voting_function(train_data, estimators, voting='soft', threshold=0.22)

---

### 모델 학습(train 데이터 전체 학습)

위의 모델학습 코드에서 함수명만 바뀌고 들어가는 값들은 동일  
-> 위의 코드 복붙한다음 함수명만 바꿔주면 사용하기 편함  

In [24]:
model_Dam = fit_all_train_data_function(
    'xgb', train_data_dam
    , n_estimators = 1244
    , learning_rate = 0.1258535425769987
    , max_depth = 26
    , alpha = 2.1820842842359597e-06
    , gamma = 0.00010809657684921935
    , reg_alpha = 0.5844029076359536
    , reg_lambda = 0.4748752246073433
    , colsample_bytree = 0.9607659760060685
    , subsample = 0.7147741317935203
    , objective = 'binary:logistic'
    , tree_method = 'exact'
    , random_state=RANDOM_STATE
)

model_AutoClave = fit_all_train_data_function(
    'xgb', train_data_autoclave,
    n_estimators = 1152, 
    learning_rate = 0.02466611382982541, 
    max_depth = 29, 
    alpha = 2.9180083404308157e-05, 
    gamma = 0.00012667501319666823, 
    reg_alpha = 0.6903592486292155, 
    reg_lambda = 0.5638873235014423, 
    colsample_bytree = 0.9432782030604233, 
    subsample = 0.19192246128663584,
    objective = 'binary:logistic',  # 이진 분류
    tree_method = "exact", 
    random_state=RANDOM_STATE
)

model_Fill1 = fit_all_train_data_function(
    'xgb', train_data_fill1,
    n_estimators = 1899, 
    learning_rate = 0.011878583548993711, 
    max_depth = 12, 
    alpha = 0.004515243354832891,
    gamma = 0.0015693650802180896,
    reg_alpha = 0.7484424912256998, 
    reg_lambda = 0.27164326303977143, 
    colsample_bytree = 0.7901385059430825,
    subsample = 0.9924662032617025,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

model_Fill2 = fit_all_train_data_function(
    'xgb', train_data_fill2,
    n_estimators = 1162, 
    learning_rate = 0.014523070494025153, 
    max_depth = 8, 
    alpha = 0.00012198482017902725, 
    gamma = 0.001236902841680112, 
    reg_alpha = 0.7331637000614692, 
    reg_lambda = 0.5237223061096699, 
    colsample_bytree = 0.8250374170841293, 
    subsample = 0.31906427054137687,
    objective = 'binary:logistic',
    tree_method = 'exact',
    random_state=RANDOM_STATE
)

# model_All = fit_all_train_data_function(
# 'xgb', train_data,
#     n_estimators = 2427,
#     learning_rate = 0.010774204513905965, 
#     max_depth = 17, 
#     alpha = 0.0005233654110538582, 
#     gamma = 5.551445919277608e-05, 
#     reg_alpha = 0.9652805882189326, 
#     reg_lambda = 0.3542856398135083, 
#     colsample_bytree = 0.9094884645797131, 
#     subsample = 0.1733751790853043,
#     objective = 'binary:logistic',  # 이진 분류
#     tree_method = "exact", 
#     random_state=RANDOM_STATE
# )

xgb 모델이 train_data_dam 데이터로 학습 완료
xgb 모델이 train_data_autoclave 데이터로 학습 완료
xgb 모델이 train_data_fill1 데이터로 학습 완료
xgb 모델이 train_data_fill2 데이터로 학습 완료
xgb 모델이 train_data 데이터로 학습 완료


---

In [25]:
# 예측에 필요한 데이터 분리
x_test_dam = test_data_dam.drop(["target", "Set ID"], axis=1)
x_test_autoclave = test_data_autoclave.drop(["target", "Set ID"], axis=1)
x_test_fill1 = test_data_fill1.drop(["target", "Set ID"], axis=1)
x_test_fill2 = test_data_fill2.drop(["target", "Set ID"], axis=1)
# x_test_all = test_data.drop(["target", "Set ID"], axis=1)

Voting(hard)

4개의 모델중 다수의 모델이 판단한 값을 타겟값으로 가져감

In [73]:
# # 예측 리스트 (하드 보팅용)
# preds = [
#     model_Dam.predict(x_test_dam)
#     , model_AutoClave.predict(x_test_autoclave)
#     , model_Fill1.predict(x_test_fill1)
#     , model_Fill2.predict(x_test_fill2)
#     , model_All.predict(x_test_all)
# ]

# # 하드 보팅 결과
# final_predictions = voting(preds, method='hard')
# print(sum(final_predictions))

Voting(soft)

4개의 모델의 확률의 합에 대한 평균이 일정 수치(스레스홀드) 값을 넘으면 AbNormal 값을 가짐  

In [27]:
# 예측 확률 리스트 (소프트 보팅용)
probs = [
    model_Dam.predict_proba(x_test_dam)[:, 1]
    , model_AutoClave.predict_proba(x_test_autoclave)[:, 1]
    , model_Fill1.predict_proba(x_test_fill1)[:, 1]
    , model_Fill2.predict_proba(x_test_fill2)[:, 1]
    # , model_All.predict_proba(x_test_all)[:, 1]
]

# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.3)
print(sum(final_predictions))

379


In [75]:
# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.28)
print(sum(final_predictions))

In [76]:
# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.26)
print(sum(final_predictions))

In [77]:
# 소프트 보팅 결과
final_predictions = voting(probs, method='soft', threshold=0.24)
print(sum(final_predictions))

=== Message ===

작성하신 답안 제출이 완료되었습니다.

Public Score : 0.22973

## 4. 제출하기


### 제출 파일 작성


In [78]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["target"] = final_predictions

# df_sub['target'] 값을 문자열 레이블로 변환
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')

# 제출 파일 저장
df_sub.to_csv("Data0827_xgb(4)_submission.csv", index=False)

In [79]:
df_sub['target'].value_counts()

In [80]:
# df_sub.head(10)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**


---

## 5. ETC...

모델을 통해 예측해 낸 정답지(csv) 파일의 결과값을 불러와 voting 하는 방식  
이때의 voting은 hard방식만 가능(예측확률은 저장되어있지 않음으로 soft 보팅은 불가능)  

In [81]:
# import pandas as pd
# import numpy as np

# def read_submission_files(file_paths):
#     """
#     제출 파일을 읽어와서 예측 결과를 반환합니다.

#     Parameters:
#     file_paths (list of str): 제출 파일 경로 리스트

#     Returns:
#     list of np.array: 각 제출 파일의 예측 결과 리스트
#     """
#     predictions = []
#     for file_path in file_paths:
#         df = pd.read_csv(file_path)
#         preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
#         predictions.append(preds)
#     return predictions

# def hard_voting(preds):
#     """
#     하드 보팅을 사용하여 최종 예측을 수행합니다.

#     Parameters:
#     preds (list of np.array): 각 모델의 예측 배열 리스트

#     Returns:
#     np.array: 최종 예측 결과
#     """
#     preds = np.array(preds)
#     final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
#     return final_predictions

submission_file 에서 csv 파일을 불러와 hard voting 하는 알고리즘  

In [82]:
# import pandas as pd

# # 공통 경로
# common_path = "C:/Users/KimDongyoung/Desktop/git_LGaimers5/Lg_aimers5/data/submission_file/"

# # 제출 파일 이름 리스트
# file_names = [
#     "data0816_light_cutoff0.28_submission.csv",
#     "submission_score0.23.csv"
#     # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
# ]

# # 경로를 추가하는 함수
# def add_common_path(file_names, common_path):
#     return [common_path + file_name for file_name in file_names]

# # 경로가 추가된 파일 리스트
# file_paths = add_common_path(file_names, common_path)

# # 제출 파일에서 예측 결과 읽어오기
# predictions = read_submission_files(file_paths)

# # 하드 보팅 결과
# final_predictions_hard = hard_voting(predictions)

# # 결과를 새로운 제출 파일로 저장할 파일 이름
# output_file_name = "final_submission.csv" # <----- 파일 이름을 변경하세요!!

# # 결과를 새로운 제출 파일로 저장
# df_sub = pd.read_csv(file_paths[0])
# df_sub["target"] = final_predictions_hard
# df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
# df_sub.to_csv(output_file_name, index=False)

# print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

In [83]:
# df_sub['target'].value_counts()

In [84]:
# df_sub.head(10)